In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [272]:
indicators = pd.read_csv("dataset/Indicators.csv")

In [273]:
indicators.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,1.335609e+02
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,8.779760e+01
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,6.634579e+00
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,8.102333e+01
4,Arab World,ARB,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,3.000000e+06


### Seleziono l'indicatore di mio interesse


In [274]:
indic_list = ['SM.POP.TOTL.ZS']
indicators = indicators[indicators.IndicatorCode.isin(indic_list)]

In [275]:
indicators.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
18,Arab World,ARB,International migrant stock (% of population),SM.POP.TOTL.ZS,1960,2.990637
99,Caribbean small states,CSS,International migrant stock (% of population),SM.POP.TOTL.ZS,1960,4.483143
184,Central Europe and the Baltics,CEB,International migrant stock (% of population),SM.POP.TOTL.ZS,1960,4.999099
278,East Asia & Pacific (all income levels),EAS,International migrant stock (% of population),SM.POP.TOTL.ZS,1960,0.853820
396,East Asia & Pacific (developing only),EAP,International migrant stock (% of population),SM.POP.TOTL.ZS,1960,0.407666


### Faccio il pivoting in modo da avere 3 colonne, Valore dell'indicatore, CountryCode e Year

In [276]:
indicators2 = indicators.pivot(columns='IndicatorCode', values=['Value','CountryCode','Year'])

In [277]:
indicators2.shape


(2491, 3)

In [278]:
indicators2.head(10)

,Value,CountryCode,Year
IndicatorCode,SM.POP.TOTL.ZS,SM.POP.TOTL.ZS,SM.POP.TOTL.ZS
18,2.99064,ARB,1960
99,4.48314,CSS,1960
184,4.9991,CEB,1960
278,0.85382,EAS,1960
396,0.407666,EAP,1960
531,3.39936,EMU,1960
846,3.69318,EUU,1960
922,3.69292,FCS,1960
1017,3.88721,HPC,1960


### Rinomino le colonne in modo coerente e più comprensibile

In [259]:
indicators2.columns = [' '.join(col).strip() for col in indicators2.columns.values]

In [260]:
indicators2.columns = [indicators2.columns[0].split()[1], 'CountryCode', 'Year']


In [261]:
indicators2.head()

,SP.DYN.AMRT.MA,CountryCode,Year
51,357.706,ARB,1960
132,261.119,CSS,1960
212,201.049,CEB,1960
311,446.838,EAS,1960
431,478.537,EAP,1960


In [262]:
indicators2.to_csv(path_or_buf='./part/mort_rateM')

### Ripeto le righe sopra PER OGNI indicatore di mio interesse. Terminata la creazione dei csv li ricarico e ne faccio il merge, qui sotto.

In [263]:
birth = pd.read_csv("./part/birth")
life = pd.read_csv("./part/arms_import")
arms = pd.read_csv("./part/life_expect")
rural = pd.read_csv("./part/rural")
urban = pd.read_csv("./part/urban")
mrF = pd.read_csv("./part/mort_rateF")
mrM = pd.read_csv("./part/mort_rateM")

In [264]:
del birth['Unnamed: 0']
del arms['Unnamed: 0']
del life['Unnamed: 0']
del urban['Unnamed: 0']
del rural['Unnamed: 0']
del mrF['Unnamed: 0']
del mrM['Unnamed: 0']

### Come vediamo ci sono delle shape diverse nei nostri DF, di conseguenza quando andremo a mergiare avremo dei NaN.

In [265]:
print('Birth: '+ str(birth.shape[0]) + '. Arms: ' + str(arms.shape[0]) + '. Life:' + str(life.shape[0]) + '. Rural:' + str(rural.shape[0]) + '. Urban:' + str(urban.shape[0]) + '. Mort rate F:' + str(mrF.shape[0])+ '. Mort rate M:' + str(mrM.shape[0]))

Birth: 12485. Arms: 12304. Life:7333. Rural:13374. Urban:13374. Mort rate F:11939. Mort rate M:11939


### Eseguiamo un primo merge, in modo da vedere la forma dell'ouput, e verificare che ci sono dei NaN in entrambe le colonne degli indicatori.

In [266]:
tot = birth.merge(arms, on = ['CountryCode', 'Year'], how = 'outer')

In [267]:
tot.head()

,SP.DYN.CBRT.IN,CountryCode,Year,SP.DYN.LE00.IN
0,47.697888,ARB,1960,46.847059
1,40.017001,CSS,1960,62.271795
2,19.117435,CEB,1960,67.823762
3,26.362350,EAS,1960,48.298317
4,26.635950,EAP,1960,45.549658


In [268]:
tot[tot['SP.DYN.LE00.IN'].isnull()].head()

,SP.DYN.CBRT.IN,CountryCode,Year,SP.DYN.LE00.IN
52,28.3,BMU,1960,NaN
117,14.1,IMY,1960,NaN
118,26.9,ISR,1960,NaN
136,23.1,LIE,1960,NaN
342,13.8,IMY,1961,NaN


In [269]:
tot[tot['SP.DYN.CBRT.IN'].isnull()].head()

,SP.DYN.CBRT.IN,CountryCode,Year,SP.DYN.LE00.IN
12485,NaN,BMU,1965,68.897805
12486,NaN,FRO,1973,74.497561
12487,NaN,FRO,1978,75.985366
12488,NaN,FRO,1986,75.265854
12489,NaN,FRO,1987,75.465854


# Eseguiamo tutti i merge

In [270]:
tot = tot.merge(life, on = ['CountryCode', 'Year'], how = 'outer')
tot = tot.merge(urban, on = ['CountryCode', 'Year'], how = 'outer')
tot = tot.merge(rural, on = ['CountryCode', 'Year'], how = 'outer')
tot = tot.merge(mrF, on = ['CountryCode', 'Year'], how = 'outer')
tot = tot.merge(mrM, on = ['CountryCode', 'Year'], how = 'outer')

In [279]:
tot

,SP.DYN.CBRT.IN,CountryCode,Year,SP.DYN.LE00.IN,MS.MIL.MPRT.KD,SP.URB.TOTL,SP.RUR.TOTL,SP.DYN.AMRT.FE,SP.DYN.AMRT.MA
0,47.697888,ARB,1960,46.847059,5.380000e+08,28937699.0,63558203.0,298.935703,357.705584
1,40.017001,CSS,1960,62.271795,NaN,1324191.0,2866619.0,201.475579,261.118888
2,19.117435,CEB,1960,67.823762,2.139000e+09,40680946.0,50720637.0,124.749727,201.048873
3,26.362350,EAS,1960,48.298317,5.344000e+09,231871389.0,799992005.0,326.366329,446.838421
4,26.635950,EAP,1960,45.549658,3.535000e+09,151665806.0,744827185.0,353.252556,478.537086
...,...,...,...,...,...,...,...,...,...
13468,NaN,SRB,1985,NaN,NaN,NaN,NaN,107.875400,199.063200
13469,NaN,SRB,1986,NaN,NaN,NaN,NaN,106.232200,201.110600
13470,NaN,SRB,1987,NaN,NaN,NaN,NaN,104.589000,203.158000
13471,NaN,SRB,1988,NaN,NaN,NaN,NaN,104.610400,201.766000


### Riga successiva da eseguire più volte per cambiare l'ordine delle colonne

In [288]:
cols = tot.columns.tolist()
cols = cols[-1:] + cols[:-1]
tot = tot[cols]
tot.head()

,CountryCode,Year,SP.DYN.LE00.IN,MS.MIL.MPRT.KD,SP.URB.TOTL,SP.RUR.TOTL,SP.DYN.AMRT.FE,SP.DYN.AMRT.MA,SP.DYN.CBRT.IN
0,ARB,1960,46.847059,5.380000e+08,28937699.0,63558203.0,298.935703,357.705584,47.697888
1,CSS,1960,62.271795,NaN,1324191.0,2866619.0,201.475579,261.118888,40.017001
2,CEB,1960,67.823762,2.139000e+09,40680946.0,50720637.0,124.749727,201.048873,19.117435
3,EAS,1960,48.298317,5.344000e+09,231871389.0,799992005.0,326.366329,446.838421,26.362350
4,EAP,1960,45.549658,3.535000e+09,151665806.0,744827185.0,353.252556,478.537086,26.635950


In [291]:
tot.to_csv(path_or_buf='./part/partialDF', index=False)

In [292]:
part = pd.read_csv("./part/partialDF")
part.head()

,CountryCode,Year,SP.DYN.LE00.IN,MS.MIL.MPRT.KD,SP.URB.TOTL,SP.RUR.TOTL,SP.DYN.AMRT.FE,SP.DYN.AMRT.MA,SP.DYN.CBRT.IN
0,ARB,1960,46.847059,5.380000e+08,28937699.0,63558203.0,298.935703,357.705584,47.697888
1,CSS,1960,62.271795,NaN,1324191.0,2866619.0,201.475579,261.118888,40.017001
2,CEB,1960,67.823762,2.139000e+09,40680946.0,50720637.0,124.749727,201.048873,19.117435
3,EAS,1960,48.298317,5.344000e+09,231871389.0,799992005.0,326.366329,446.838421,26.362350
4,EAP,1960,45.549658,3.535000e+09,151665806.0,744827185.0,353.252556,478.537086,26.635950
